In [1]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AY0e-g6uQNiurf7HPap_iTh-pNx2pFbLQFs361Yu89juAezNm6EGWfNyT-w

Successfully saved authorization token.


In [2]:
from geetools import tools, utils
import geedatasets
import utils
import pandas as pd
import altair as alt
import numpy as np
import geopandas as gpd
import folium
from geopandas import GeoDataFrame
from shapely.geometry import Point,Polygon
#import numpy
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt

In [4]:
collectionName_L5 = "LANDSAT/LT05/C02/T2_L2" 

In [3]:
#NDVI

### Landsat
def NDVI_L5(image):
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI').set('system:time_start', image.get('system:time_start'))
    return image.addBands(ndvi)


#-----------------------------------------------------------------------------------------------------------------#
def NDMI_L5(image):
    ndmi = image.normalizedDifference(['B4', 'B5']).rename('NDMI').set('system:time_start', image.get('system:time_start'))
    return image.addBands(ndmi)


#-----------------------------------------------------------------------------------------------------------------#

def NDWI_L5(image):
    ndwi = image.normalizedDifference(['B2', 'B4']).rename('NDWI').set('system:time_start', image.get('system:time_start'))
    return image.addBands(ndwi)


#-----------------------------------------------------------------------------------------------------------------#

# Formula of MSI = MidIR / NIR
# MSI (Landsat 4 – 7) = B5 / B4

def MSI_L5(image):
    return ee.Image(0).expression(
      'swir/ nir',
      {
          'nir': image.select('B4'),
          'swir': image.select('B5'),
      }).set('system:time_start', image.get('system:time_start')).rename('MSI')



#-----------------------------------------------------------------------------------------------------------------#
#NDII =(NIR −SWIR1)/(NIR +SWIR1)
def NDII_L5(image):
    return ee.Image(0).expression(
      '(nir-swir)/ (nir+swir)',
      {
          'nir': image.select('B4'),
          'swir': image.select('B5'),
      }).set('system:time_start', image.get('system:time_start')).rename('NDII')




#-----------------------------------------------------------------------------------------------------------------#
          
def SAVI_L5(image):
  """A function to compute Soil Adjusted Vegetation Index."""
  return ee.Image(0).expression(
      'Gain * float(nir - red)/ (nir + red + L)',
      {
          'Gain': ee.Number(1.5),
          'nir': image.select('B4'),
          'red': image.select('B3'),
          'L': 5000
      }).set('system:time_start', image.get('system:time_start')).rename('SAVI')


#-----------------------------------------------------------------------------------------------------------------#



# In Landsat 4-7, EVI = 2.5 * ((Band 4 – Band 3) / (Band 4 + 6 * Band 3 – 7.5 * Band 1 + 1)).
#"""A function to compute Enhanced Vegetation Index."""
def EVI_L5(image):
    return ee.Image(0).expression(
      'Gain*((NIR-R)/(NIR + C1*R - C2*B + L))',
      {
      'Gain': ee.Number(2.5),
      'B': image.select('B1'),
      'R': image.select('B3'),
      'NIR': image.select('B4'),
      'L': ee.Number(1),
      'C1': ee.Number(6),
      'C2': ee.Number(7.5)
      }).set('system:time_start', image.get('system:time_start')).rename('EVI')



#-----------------------------------------------------------------------------------------------------------------#
#In Landsat 4-7, MSAVI = (2 * Band 4 + 1 – sqrt ((2 * Band 4 + 1)2 – 8 * (Band 4 – Band 3))) / 2.
def MSAVI_L5(image):
    return ee.Image(0).expression(
      '(Gain * NIR + L - sqrt(pow((Gain * NIR + L), 2) - C * (NIR - R)) ) / Gain',
      {
      'Gain': ee.Number(2),
      'R': image.select('B3'),
      'NIR': image.select('B4'),
      'L': ee.Number(1),
      'C': ee.Number(8)
      }).set('system:time_start', image.get('system:time_start')).rename('MSAVI')



#-----------------------------------------------------------------------------------------------------------------#

#The formula of NDWI modified by Xu (2005).
#It uses Green and SWIR band.
# MNDWI = (Green – SWIR) / (Green + SWIR)
def MNDWI_L5(image):
    mndwi = image.normalizedDifference(['B2', 'B5']).rename('MNDWI').set('system:time_start', image.get('system:time_start'))
    return image.addBands(mndwi)

#-----------------------------------------------------------------------------------------------------------------#
def SWIR_L5(image):
    swir = image.select('B5').rename('SWIR').set('system:time_start', image.get('system:time_start'))
    return image.addBands(swir)


#-----------------------------------------------------------------------------------------------------------------#
#Mask cloud and Shadow for Landsat images L5,7


def cloudMaskL457(image):
    qa = image.select('pixel_qa')
    #If the cloud bit (5) is set and the cloud confidence (7) is high
    #or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))
    #Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)



# Look for the bitmasks in
# https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT05_C02_T2_L2?hl=en
def mask_water(image):
    #Get the pixel QA band.
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(2).neq(0)
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(mask).updateMask(mask2)

# def mask_water2(image):
#     qa = image.select('pixel_qa')
#     mask = qa.bitwiseAnd(1<< 2)
#     #Remove edge pixels that don't occur in all bands
#     mask2 = image.mask().reduce(ee.Reducer.min())
#     return image.updateMask(mask.Not()).updateMask(mask2)

def mask_snow(image):
    qa = image.select('pixel_qa')
    cloud = qa.bitwiseAnd(1 << 4)
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)


# def mask_snow(image):
#     #Get the pixel QA band.
#     qa = image.select('pixel_qa')
#     snowBitMask = (1 << 4)
#     mask = qa.bitwiseAnd(snowBitMask).eq(0);
#     mask2 = image.mask().reduce(ee.Reducer.min())
#     return image.updateMask(mask).updateMask(mask2)

In [6]:
def run_export(image, crs, filename, scale, region, maxPixels=1e13):
    '''
    Runs an export function on GEE servers
    '''
    task_config = {'fileNamePrefix': filename,'crs': crs,'scale': scale,'maxPixels': maxPixels,'fileFormat': 'GeoTIFF','region': region,}
    task = ee.batch.Export.image.toDrive(image, filename, **task_config)
    task.start()

In [7]:
def gee_geometry_from_shapely(geom, crs='epsg:4326'):
    """ 
    Simple helper function to take a shapely geometry and a coordinate system and convert them to a 
    Google Earth Engine Geometry.
    """
    from shapely.geometry import mapping
    ty = geom.type
    if ty == 'Polygon':
        return ee.Geometry.Polygon(ee.List(mapping(geom)['coordinates']), proj=crs, evenOdd=False)
    elif ty == 'Point':
        return ee.Geometry.Point(ee.List(mapping(geom)['coordinates']), proj=crs, evenOdd=False)    
    elif ty == 'MultiPolygon':
        return ee.Geometry.MultiPolygon(ee.List(mapping(geom)['coordinates']), proj=crs, evenOdd=False)

**system:time_start**
<br>The Earth Engine time stamp in milliseconds since the UNIX epoch. See this link (https://en.wikipedia.org/wiki/Unix_time) for more information. The time stamp is set to the nominal image acquisition time for single scenes. It is set to the nominal composite start period for temporal composites.

In [8]:
#Returns the difference between two Dates in the specified units; 
#the result is floating-point and based on the average length of the unit.
#ee.Date.difference(start, unit)
# Arguments:
# this:date (Date)
# start (Date)
# unit (String):
# One of 'year', 'month' 'week', 'day', 'hour', 'minute', or 'second'.
#Returns: Float
    
    
def addDate(image):
        date = ee.Date(image.get('system:time_start'))
        years = date.difference(ee.Date('1984-01-01'), 'year');
        return image.addBands(ee.Image.constant(years)).addBands(ee.Image(years).rename('t')).float()

In [9]:
#Load Image Collection
collection_L5 = ee.ImageCollection(collectionName_L5)

In [10]:
# Define the area of interest

from shapely.geometry import Polygon

geom = Polygon([[-122.7609625539781,71.0248652750595],
                [-121.7502203664781,71.35078807875405],
                [-121.0031500539781,71.37185539607945],
                [-120.3219977102281,71.53956940315283],
                [-120.3000250539781,71.98650637573677],
                [-119.8825445852281,72.18251814893772],
                [-119.2233648977281,72.33650688912165],
                [-119.0695563039781,72.62751123507817],
                [-118.3664313039781,72.77777272011134],
                [-117.4216070852281,73.02327346292203],
                [-116.7624273977281,73.12562314751086],
                [-115.6637945852281,73.35372393902468],
                [-115.1803961477281,73.49164105978065],
                [-117.2238531789781,74.21859485977957],
                [-118.1247320852281,74.30204109643341],
                [-118.8498297414781,74.23054216680322],
                [-119.5749273977281,74.24844660046523],
                [-121.2448492727281,74.56735766699148],
                [-121.7941656789781,74.57904742715868],
                [-122.3654547414781,74.4970373362395],
                [-124.8483648977281,74.36730409994937],
                [-124.6066656789781,74.1226983323726],
                [-124.1672125539781,73.81934897680587],
                [-124.0793219289781,73.57882746350154],
                [-124.4528570852281,73.39772768318414],
                [-124.8263922414781,73.21468747419634],
                [-124.9802008352281,72.96543724049849], 
                [-125.3097906789781,72.81676365632822],
                [-125.1999273977281,72.620949382208],
                [-125.9469977102281,72.08139173142834],
                [-125.8371344289781,71.9116129732407],
                [-125.3976813039781,71.93206824488587],
                [-124.4308844289781,71.71961482925289],
                [-123.9914313039781,71.56737970042492],
                [-123.7057867727281,71.39289975804704],
                [-123.4421148977281,71.06055580588858],
                [-122.7609625539781,71.0248652750595] ])
search_area = gee_geometry_from_shapely(geom)

In [11]:
# Filter the collections
collection_5 = ee.ImageCollection(collection_L5).filter(ee.Filter.date('1984-01-01', '2014-01-01')
     .calendarRange(6,8, 'month'))\
    .filterBounds(search_area)\
    .filter(ee.Filter.lt('CLOUD_COVER', 30))\
    .map(cloudMaskL457)\
    .map(mask_water)\
    .map(mask_snow)\
    .sort('system:time_start')

In [12]:
#Calculate NDVI and and add Date band(=doy)
NDVI_5=collection_5.map(NDVI_L5)
NDVI_5_date=NDVI_5.map(addDate);


#Calculate EVI and and add Date band(=doy)
EVI_5 = collection_5.map(EVI_L5)
EVI_5_date = EVI_5.map(addDate);


#Calculate MSI and and add Date band(=doy)
MSI_5 = collection_5.map(MSI_L5)
MSI_5_date = MSI_5.map(addDate);


#Calculate NDII and and add Date band(=doy)
NDII_5 = collection_5.map(NDII_L5)
NDII_5_date = NDII_5.map(addDate);


# #Calculate SAVI and and add Date band(=doy)
SAVI_5 = collection_5.map(SAVI_L5)
SAVI_5_date = SAVI_5.map(addDate);


#Calculate MSAVI and and add Date band(=doy)
MSAVI_5 = collection_5.map(MSAVI_L5)
MSAVI_5_date = MSAVI_5.map(addDate);


#Calculate NDMI and and add Date band(=doy)
NDMI_5 = collection_5.map(NDMI_L5)
NDMI_5_date = NDMI_5.map(addDate);


#Calculate NDWI and and add Date band(=doy)
NDWI_5 = collection_5.map(NDWI_L5)
NDWI_5_date = NDWI_5.map(addDate);


#Calculate MNDWI and add Date band(=doy)
MNDWI_5 = collection_5.map(MNDWI_L5)
MNDWI_5_date = MNDWI_5.map(addDate);


#Short Wave
SWIR_5 = collection_5.map(SWIR_L5);
SWIR_5_date = SWIR_5.map(addDate);

**ee.Reducer.linearFit()**
<br>Computes the least squares estimate of a linear function of one variable with a constant term.
<br>The data should be set up as a two-band input image, where the first band is the independent variable (time) and the <br>second band is the dependent variable. 
<br>https://developers.google.com/earth-engine/guides/reducers_regression

In [13]:
# Calculate the linear fit 
linear_fit_NDVI = NDVI_5_date.select(['t', 'NDVI']).reduce(ee.Reducer.linearFit()).select(['scale', 'offset'])
linear_fit_EVI = EVI_5_date.select(['t', 'EVI']).reduce(ee.Reducer.linearFit()).select(['scale', 'offset'])
linear_fit_MSI = MSI_5_date.select(['t', 'MSI']).reduce(ee.Reducer.linearFit()).select(['scale', 'offset'])

linear_fit_NDII = NDII_5_date.select(['t', 'NDII']).reduce(ee.Reducer.linearFit()).select(['scale', 'offset'])
linear_fit_SAVI = SAVI_5_date.select(['t', 'SAVI']).reduce(ee.Reducer.linearFit()).select(['scale', 'offset'])
linear_fit_MSAVI = MSAVI_5_date.select(['t', 'MSAVI']).reduce(ee.Reducer.linearFit()).select(['scale', 'offset'])

linear_fit_NDMI = NDMI_5_date.select(['t', 'NDMI']).reduce(ee.Reducer.linearFit()).select(['scale', 'offset'])
linear_fit_NDWI = NDWI_5_date.select(['t', 'NDWI']).reduce(ee.Reducer.linearFit()).select(['scale', 'offset'])

linear_fit_MNDWI = MNDWI_5_date.select(['t', 'MNDWI']).reduce(ee.Reducer.linearFit()).select(['scale', 'offset'])
linear_fit_SWIR = SWIR_5_date.select(['t', 'SWIR']).reduce(ee.Reducer.linearFit()).select(['scale', 'offset'])


In [14]:
#NDVI_5_7_8_date.getInfo()

In [15]:
#Export the images to google drive

In [16]:
run_export(linear_fit_NDVI, 'epsg:4326', 'Lin_NDVI_1984_2011_L5_30Cl', scale=30, region=search_area, maxPixels=1e13)

In [ ]:
run_export(linear_fit_MSI, 'epsg:4326', 'lin_MSI_1984_2011_L5_30Cl', scale=30, region=search_area, maxPixels=1e13)

In [ ]:
run_export(linear_fit_EVI, 'epsg:4326', 'Lin_EVI_1984_2011_L5_30Cl', scale=30, region=search_area, maxPixels=1e13)

In [ ]:
run_export(linear_fit_NDII, 'epsg:4326', 'Lin_NDII_1984_2011_L5_30Cl', scale=30, region=search_area, maxPixels=1e13)

In [ ]:
run_export(linear_fit_SAVI, 'epsg:4326', 'Lin_SAVI_1984_2011_L5_30Cl', scale=30, region=search_area, maxPixels=1e13)

In [ ]:
run_export(linear_fit_NDMI, 'epsg:4326', 'Lin_NDMI_1984_2011_L5_30Cl', scale=30, region=search_area, maxPixels=1e13)

In [ ]:
run_export(linear_fit_SWIR, 'epsg:4326', 'Lin_SWIR_1984_2011_L5_30Cl', scale=30, region=search_area, maxPixels=1e13)

In [ ]:
run_export(linear_fit_MSAVI, 'epsg:4326', 'Lin_MSAVI_1984_2011_L5_30Cl', scale=30, region=search_area, maxPixels=1e13)

In [ ]:
run_export(linear_fit_NDWI, 'epsg:4326', 'Lin_NDWI_1984_2011_L5_30Cl', scale=30, region=search_area, maxPixels=1e13)

In [ ]:
run_export(linear_fit_MNDWI, 'epsg:4326', 'Lin_MNDWI_1984_2011_L5_30Cl', scale=30, region=search_area, maxPixels=1e13)

In [ ]:
def dates_available(geCollection):
    """
    Returns a list of the dates available for this collection.
    geCollection: ee.ImageCollection object
    Returns a list of date strings in YYYY-MM-DD format.
    """

    timestamps =  geCollection.aggregate_array('system:time_start').getInfo()
    dateformat_array = [timestamp_to_datetime(t) for t in timestamps]

    return  dateformat_array

In [ ]:
def timestamp_to_datetime(timestamp, time_format = '%Y-%m-%d'):
    """
    Converts the UNIX epoch timestamp used by the Earth Engine database into a
    format readable by humans (and also the format used by the Earth Engine
    date filters)
    timestamp: UNIX time epoch
    time_format: optional datetime format
    Returns a datetime string.
    """
    return datetime.datetime.fromtimestamp(timestamp/1000).strftime(time_format)

In [ ]:
def collection_length(image_collection):
    return image_collection.size().getInfo()

In [ ]:
# import datetime
# # which dates are available in a collection.
# dates_of_images = dates_available(SWIR_5_date)
# print ("Dates available:",dates_of_images)

In [ ]:
def available_bands(image_collection):
    """
    Determines which bands are available in the image collection.
    Since the images in a specific collection are not guarenteeded to all
    share the same bands, this function looks at the bands in the first image, and
    then calculates how many images in the collection include that band.
    Since the band names are determined from a single image, the bands that
    are returned may only be a subset of all the bands present in a collection.
    However, this function is helpful to at least determine the naming scheme
    used in a collecion, and whether it is safe to assume that certain bands
    are included in every image in the collection.
    image_collection: ee.ImageCollection object
    Hits the server 1+number_of_bands times
    Returns a dictionary in format
    { "band1": { "number_available" : number of images that contain band1,
                 "percent_available" : percent of all images in collection that contain band1 }
                },
      "band2": {...
    }
    """
    band_ids = [band_info['id']
                for band_info
                in image_collection.first().getInfo()['bands']]

    collection_size = image_collection.size().getInfo()

    availability_dict = {}
    for b in band_ids:
        imgs_available = image_collection.select(b).size().getInfo()
        percent_available = imgs_available/collection_size*100
        availability_dict[b] = {
            "number_available": imgs_available,
            "percent_available": percent_available
        }
        # print "'"+b+"' available in "+ str(imgs_available) + " images. ("+str(percent_available)+"%)"

    return availability_dict

In [ ]:
def date_slices(geImageCollection, bounds_geometry, descending=False):
    """
    Returns a list of non-overlapping date ranges where every date range
    covers the bounds_geometry.
    This function is not very efficient takes a loong time.
    """

    date_slices = []
    date_list = list(set(dates_available(geImageCollection)))
    date_list.sort()
    print (len(date_list), "unique dates found.")
    # convert these strings into Date objects
    date_list = [ee.Date(d) for d in date_list]
    start_date = date_list[0]

    for i in range(len(date_list)):
        # We're using an integer iterator because we want some lookahead later on
        end_date = date_list[i].advance(1, 'day')
        potential_slice_collection = geImageCollection.filter(ee.Filter.date(start_date, end_date))

        # If this date slice covers the image 100%, add it to the date_slices list
        # and increment the start date to the next available date
        if collection_fills_bounds(potential_slice_collection, bounds_geometry):
            date_slices.append((start_date, end_date))
            try:
                start_date = date_list[i+1]
            except IndexError:
                pass

        #print (i)

    return date_slices